In [7]:
import os

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf

## Preparing Data

In [9]:
# read the training data
data = pd.read_csv('train.csv')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [11]:
# clean the data
data['Sex'] =  data['Sex'].apply(lambda s: 1 if s == 'male' else 0)
data = data.fillna(0)
dataset_X = data[['Sex', 'Age', 'Pclass', 'SibSp', 'Parch', 'Fare']]
dataset_X = dataset_X.as_matrix()

In [12]:
# one-hot encoding the label
data['Deceased'] = data['Survived'].apply(lambda s: int(not s))
dataset_Y = data[['Deceased', 'Survived']]
dataset_Y = dataset_Y.as_matrix()

In [13]:
# split training and validation set
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    dataset_X, dataset_Y, test_size=0.2, random_state=42)

## Constructing Dataflow Graph

In [14]:
# arguments that can be set in command line
tf.app.flags.DEFINE_integer('epochs', 10, 'Training epochs')
tf.app.flags.DEFINE_integer('batch_size', 10, 'size of mini-batch')
FLAGS = tf.app.flags.FLAGS

In [15]:
with tf.name_scope('input'):
    # create symbolic variables
    # the None means we can put any number of records for training
    X = tf.placeholder(tf.float32, shape=[None, 6])
    y_true = tf.placeholder(tf.float32, shape=[None, 2])

In [16]:
with tf.name_scope('classifier'):
    # weights and bias are the variables to be trained
    weights = tf.Variable(tf.random_normal([6, 2]))
    bias = tf.Variable(tf.zeros([2]))
    
    # softmax is the multiclass logistic regression
    y_pred = tf.nn.softmax(tf.matmul(X, weights) + bias)

    # add histogram summaries for weights, view on tensorboard
    tf.summary.histogram('weights', weights)
    tf.summary.histogram('bias', bias)

In [17]:
# Minimise cost using cross entropy
# NOTE: add a epsilon(1e-10) when calculate log(y_pred),
# otherwise the result will be -inf
with tf.name_scope('cost'):
    cross_entropy = - tf.reduce_sum(y_true * tf.log(y_pred + 1e-10),
                                    reduction_indices=1)
    cost = tf.reduce_mean(cross_entropy)
    tf.summary.scalar('loss', cost)

In [18]:
# use gradient descent optimizer to minimize cost
with tf.name_scope('optimizer'):
    train_op = tf.train.GradientDescentOptimizer(0.001).minimize(cost)

In [19]:
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(y_true, 1), tf.argmax(y_pred, 1))
    acc_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    # Add scalar summary for accuracy
    tf.summary.scalar('accuracy', acc_op)

In [20]:
global_step = tf.Variable(0, name='global_step', trainable=False)

In [21]:
# use saver to save and restore model
saver = tf.train.Saver()

# this variable won't be stored, since it is declared after tf.train.Saver()
non_storable_variable = tf.Variable(777)

In [22]:
ckpt_dir = './ckpt_dir'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

## Training The Model

In [ ]:
# use session to run the calculation
with tf.Session() as sess:
    # create a log writer. run 'tensorboard --logdir=./logs'
    writer = tf.summary.FileWriter('./logs', sess.graph)
    merged = tf.summary.merge_all()

    # variables have to be initialized at the first place
    tf.global_variables_initializer().run()

    # restore variables from 'checkpoint' file if exists ('checkpoint' stores the latest saved checkpoint)
    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print('Restoring from checkpoint: %s' % ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
        
    # get the last global step, t.eval() same as tf.get_default_session().run(t), eval the tensor
    start = global_step.eval()
    
    # training loop
    for epoch in range(start, start + FLAGS.epochs):
        total_loss = 0.
        for i in range(0, len(X_train), FLAGS.batch_size):
            # train with mini-batch
            feed_dict = {
                X: X_train[i: i + FLAGS.batch_size],
                y_true: y_train[i: i + FLAGS.batch_size]
            }
            # fetches and feed_dict
            _, loss = sess.run([train_op, cost], feed_dict=feed_dict)
            total_loss += loss
        # display loss per epoch
        print('Epoch: %04d, loss=%.9f' % (epoch + 1, total_loss))

        summary, accuracy = sess.run([merged, acc_op],
                                     feed_dict={X: X_val, y_true: y_val})
        writer.add_summary(summary, epoch)  # Write summary
        print('Accuracy on validation set: %.9f' % accuracy)

        # set and update(eval) global_step with epoch
        global_step.assign(epoch).eval()
        saver.save(sess, ckpt_dir + '/logistic.ckpt',
                   global_step=global_step)
    print('Training complete!')

In [78]:
# check the parameters in checkpoint
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
print_tensors_in_checkpoint_file(file_name='./ckpt_dir/logistic.ckpt-5', tensor_name='', all_tensors=True)

In [24]:
# check the nodes in dataflow graph
g = tf.get_default_graph()
print g.get_operations()

[<tf.Operation 'input/Placeholder' type=Placeholder>, <tf.Operation 'input/Placeholder_1' type=Placeholder>, <tf.Operation 'classifier/random_normal/shape' type=Const>, <tf.Operation 'classifier/random_normal/mean' type=Const>, <tf.Operation 'classifier/random_normal/stddev' type=Const>, <tf.Operation 'classifier/random_normal/RandomStandardNormal' type=RandomStandardNormal>, <tf.Operation 'classifier/random_normal/mul' type=Mul>, <tf.Operation 'classifier/random_normal' type=Add>, <tf.Operation 'classifier/Variable' type=VariableV2>, <tf.Operation 'classifier/Variable/Assign' type=Assign>, <tf.Operation 'classifier/Variable/read' type=Identity>, <tf.Operation 'classifier/zeros' type=Const>, <tf.Operation 'classifier/Variable_1' type=VariableV2>, <tf.Operation 'classifier/Variable_1/Assign' type=Assign>, <tf.Operation 'classifier/Variable_1/read' type=Identity>, <tf.Operation 'classifier/MatMul' type=MatMul>, <tf.Operation 'classifier/add' type=Add>, <tf.Operation 'classifier/Softmax' 